In [1]:
import torch
import torch.nn as nn
print("Using GPU: " + str(torch.cuda.is_available()))
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np

Using GPU: True


In [2]:
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
params = model.state_dict()
embeddings = params['transformer.wte.weight']

In [7]:
#Take a model's embeddings and invert x*100 percent of the elements
def invert_embeddings(embeddings, x):

    dimension = len(embeddings[0])
    chooseCount = int(dimension * x)
    #I couldn't figure out how to vectorize across this dimension
    for i in range(len(embeddings)):
        embeddings[i, np.random.choice(range(dimension), size = chooseCount, replace = False)] *= -1
    
    return embeddings

In [4]:
def add_normal_noise(original):
    observed_average_cosine_similarity = 1

    #Determines values of |x|
    original_norms = torch.linalg.norm(original, dim = 1)
    original_norms = torch.reshape(original_norms, (-1, 1))
    
    embeddings = None
    #our function to determine cosine similarity
    cos = nn.CosineSimilarity(dim=1, eps=1e-8)

    #while observed_average_cosine_similarity > 0.99:
    #to tune
    variance = 0.8

    #Copies and noises embeddings to get x'
    embeddings = original.detach().clone()
    embeddings[:,:] += np.random.normal(0, variance, embeddings.shape)
    
    #normalize to get x''
    embeddings_norms = torch.linalg.norm(embeddings, dim = 1)
    embeddings_norms = torch.reshape(embeddings_norms, (-1, 1))
    scaling_terms = original_norms / embeddings_norms
    embeddings *= scaling_terms

    cosine_similarities = cos(original, embeddings)
    cosine_similarities = cosine_similarities.detach().numpy()
    observed_average_cosine_similarity = np.mean(cosine_similarities)
    print(observed_average_cosine_similarity)
    print("")

    return embeddings
    



In [12]:
#embeddings = add_normal_noise(embeddings)
params['transformer.wte.weight'] = invert_embeddings(embeddings, .1)

In [13]:
model.load_state_dict(params)
model.save_pretrained('weights/noisedInverted10')